In [ ]:
# docker run -it -p 8050:8050 --rm scrapinghub/splash

In [7]:
import requests  # 导入网络请求模块
from urllib.parse import quote   # 导入quote方法
from bs4 import BeautifulSoup  # 导入BeautifulSoup库
import pandas as pd
import time               # 导入时间模块
import random             # 导入随机模块
from requests_html import UserAgent    # 导入HTMLSession类
ua = UserAgent().random          # 创建随机请求头

In [8]:
def processing(strs):
    s = ''  # 定义保存内容的字符串
    for n in strs:
        n = ''.join(n.split())  # 去除空字符
        s = s + n  # 拼接字符串
    return s      # 返回拼接后的字符串

In [9]:
def get_html(url):
    lua_script = '''
    function main(splash)
        splash:go("{}")
        splash:wait(1)
        return splash:html()
    end
    '''.format(url)
    splash_url = 'http://localhost:8050/execute?lua_source='+ quote(lua_script)   # Splash的execute接口地址
    # 定义headers信息
    headers = {'User-Agent':ua}
    response = requests.get(splash_url,headers=headers)       # 发送网络请求
    #print(response.text)                                      # 打印渲染后的html代码
    soup = BeautifulSoup(response.text, features="lxml") #指定解析器为lxml
    time.sleep(random.randint(6,10))          # 等待1至3秒随机时间
    return soup

In [10]:
def count_page(url):
    beautifulSoupPage = get_html(url)
    countSentence = beautifulSoupPage.find('div', class_='cnn-search__results-count').text
    countCampany = countSentence.split(' ')[5]
    return int(countCampany)

In [11]:
def get_info(url):
    beautifulSoupInfo=get_html(url)
    div_all = beautifulSoupInfo.find_all('div', class_='cnn-search__result cnn-search__result--article')
    infos = []
    for div in div_all:
        key = {}
        key['title']=div.find('div', class_='cnn-search__result-contents').h3.a.text
        time=div.find('div',class_='cnn-search__result-publish-date').text
        key['time']=processing(time)
        content=div.find('div',class_='cnn-search__result-body').text
        key['content']=content.replace('\n', '').strip()
        infos.append(key)
    return infos

In [12]:
def get_company(name,count):
    df= pd.DataFrame({'title':[],'time':[],'content':[]})
    for i in range(0,count, 50):    # 每页25为间隔，实现循环，共10页
    # 通过format替换切换页码的url地址
        try:
            #url="https://edition.cnn.com/search?q=apple&size=10&from={0}&page={1}".format((i-1)*10,i)
            url="https://edition.cnn.com/search?q={0}&size=50&from={1}".format(name,i)
            data=get_info(url)
            df=df.append(data)
            print(i)
        except Exception as e:
            print(e)
    df.to_csv("cnn_{}.csv".format(name))

In [20]:
all_company=pd.read_csv('constituents_csv.csv')
for i in all_company['Name'][100:]:
    i=i.replace(' ','%20')
    url="https://edition.cnn.com/search?q={0}&size=50&from=1".format(i)
    countCompany=count_page(url)
    print(countCompany)
    get_company(i,countCompany)
    print(i)

9
0
CF%20Industries
11
0
Charles%20River%20Laboratories
7
0
Charles%20Schwab%20Corporation


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))